In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function, division
import six, sys
sys.path.append('..'); import pathcustomize, about
about.about()

import waldo

Python 2.7.9 (default, Apr 14 2015 12:54:25) [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux2, Host: cody


In [33]:
import pathlib

import pandas as pd
import numpy
import matplotlib.pyplot as plt
import matplotlib as mpl

from waldo.conf import settings


In [7]:
waldo_directory = pathlib.Path('/home/projects/worm_movement/Data/WALDO_Data/20150515_124736/')
for i in waldo_directory.glob('*.csv'):
    print(i)

/home/projects/worm_movement/Data/WALDO_Data/20150515_124736/20150515_124736-tract_initiation.csv
/home/projects/worm_movement/Data/WALDO_Data/20150515_124736/20150515_124736-track_counts.csv
/home/projects/worm_movement/Data/WALDO_Data/20150515_124736/20150515_124736-tract_termination.csv
/home/projects/worm_movement/Data/WALDO_Data/20150515_124736/20150515_124736-network_overview.csv


In [74]:
defs = {'t': 'time from start of recording in seconds ',
        'f': 'frames since start of recording', 
        'frame': 'frames since start of recording', 
        'time': 'time from start of recording in seconds',
        
        'bid': 'blob ID number',
        'node_id': 'ID number for a track/node in WALDOs network',
        'components': 'the blob IDs that were combined to form this track',
        '# components': 'the number of tracks combined to form this track',

        
        'x': 'centroid x position', 
        'y': 'centroid y position',
        'bl': 'body lengths',

        't0': 'track start time', 
        'tN': 'track end time', 
        'f0': 'track start frame', 
        'fN': 'track end frame', 
        'x0': 'track start x coordinate', 
        'xN': 'track end x coordinate', 
        'y0': 'track start y coordinate', 
        'yN': 'track end y coordinate',        
       
        'true-pos': 'number of true positives',
        'false-pos': 'number of false positives',
        'true-neg': 'number of true negatives',
        'false-neg': 'number of false negatives',
        
        'area_median': 'the median number of pixels in blob over time (pxls ** 2)',
        'midline_median': 'the median length of blob midline over time (pxls)',
                
        'x_min': 'the minimum value of the x coordinate for this track',
        'x_max': 'the maximum value of the x coordinate for this track',
        'y_min': 'the minimum value of the y coordinate for this track',
        'y_max': 'the maximum value of the y coordinate for this track',
        'xmin': 'the minimum value of the x coordinate for this track',
        'xmax': 'the maximum value of the x coordinate for this track',
        'ymin': 'the minimum value of the y coordinate for this track',
        'ymax': 'the maximum value of the y coordinate for this track',
        
        'id': 'an ID assigned to the missing object',
        'next': 'the ID of an object that appears in this location at the next available image',
        
        
        'wm_0min': 'total worm-minutes that are in recording',
        'wm_10min': 'worm-minutes that are in tracks longer than 10 minutes',
        'wm_20min': 'worm-minutes that are in tracks longer than 20 minutes',
        'wm_30min': 'total worm-minutes that are in tracks longer than 30 minutes',
        'wm_40min': 'total worm-minutes that are in tracks longer than 40 minutes',
        'wm_50min': 'total worm-minutes that are in tracks longer than 50 minutes',
        
        '>10min': 'total number of tracks that are longer than 10 minutes',
        '>30min': 'total number of tracks that are longer than 20 minutes',
        '>20min': 'total number of tracks that are longer than 30 minutes',
        '>40min': 'total number of tracks that are longer than 40 minutes',
        '>50min': 'total number of tracks that are longer than 50 minutes',

        'moving-nodes': 'the number of nodes that move at least 1 body-length',
        'duration-med': 'the median track duration',
        'giant-component-size': "the number of nodes in the network's giant component",
        'connected-nodes': "the number of nodes in the network with at least in or out-going arc",
        'total-nodes': "the number of nodes in the network",
        'isolated-nodes': 'the number of nodes with no arcs connecting them to other nodes',

        'inside_roi': 'this is true if the blob is inside the region of interest',
        'good': "this is true if the blob was found during WALDO's image analysis",

        'unknown': 'the number of tracks that start/end for no clear reason',
        'id_change': 'the number of tracks that start/end because of interactions with other blobs',
        'outside-roi': 'the number of tracks that start/end because they move outside of the ROI',
        'split': 'the number of tracks that start/end as the result of a split',
        'join': 'the number of tracks that start/end as the result of a split',
        'on_edge': 'the number of tracks that start/end because they touch the edge of the image', 
        'timing': 'the number of tracks that start/end because the recording starts or ends',
        'reason': 'the most likely reason for losing/finding this track',

        'phase': 'the current ',
        'step': 'the ',
        'duration-std': 'the standard deviation of track durations',
        'duration-mean': 'the mean of all track durations',
        'bl_moved': 'the number of body-lengths a blob has moved',
        'lifespan': 'the number of frames that the track has existed',
        'lifespan_t': 'the number of seconds a track has existed',

        
        
}
print('---   Undefined ---')
undefined_column_names

---   Undefined ---


['roi', 'Unnamed: 0']

In [77]:
undefined_column_names = []
non_typemaped = []


type_map = {'int64':'integer', 'float64':'float', 'object': 'string', 'bool': 'boolean'}


for i in waldo_directory.glob('waldo/*.csv'):
    name = i.name
    print('\n\n')
    print(name.split('-')[-1].split('.csv')[0])
    df = pd.read_csv(str(i))
    
    # record all column names I have not yet defined
    for col in df.columns:
        if col not in defs:
            undefined_column_names.append(col)
    # record all column names I have not yet defined
    for dt in df.dtypes:
        if str(dt) not in type_map:
            print(str(dt))
            non_typemaped.append(dt)    
    print(df.head())      
    
    
    # make a table
    table_data = {'columns':list(df.columns),
                  'data type': [type_map.get(str(i), '?') for i in df.dtypes],
                  'description': [defs.get(i, 'blah') for i in df.columns],
                  }
    
    
    tdf = pd.DataFrame(table_data).set_index('columns')
    
    if 'Unnamed: 0' in tdf.index:
        tdf.drop('Unnamed: 0')
    tdf.to_latex('{name}_table.tex'.format(name=name), longtable=True)
    
undefined_column_names = list(set(undefined_column_names))




sizes
   bid  area_median  midline_median
0    1        834.0       98.423068
1    2        198.0       38.417584
2    3        657.5       95.377821
3    4        934.0      105.491261
4    5        918.0       97.744083



bounds
   bid    x_min     x_max     y_min     y_max
0    1  384.762   445.211   597.263  1144.974
1    2  405.307   409.214   564.357   565.938
2    3  510.439   511.003  1595.869  1595.897
3    4  611.240   986.966   674.165   990.875
4    5  393.888  1025.385   201.041   381.114



accuracy
   false-neg  false-pos  frame      time  true-pos
0          5          1      1     0.000         5
1          3          2   4496   300.046         6
2          3          0   8987   600.047         6
3          1          0  13481   900.082         8
4          2          0  17975  1200.116         7



missing
   id  f  t            x            y  xmin  ymin  xmax  ymax next
0  m0  1  0   524.211233  1589.669109   478  1564   568  1613     
1  m1  1  0   708.132850  